In [1]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.model_selection import train_test_split
dataset1 = pd.read_csv('../Data/data_preprocessed/ProcessDataSet1.csv')
dataset1.label.replace(-1,0,inplace=True) 
dataset2 = pd.read_csv('../Data/data_preprocessed/ProcessDataSet2.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('../Data/data_preprocessed/ProcessDataSet3.csv')
# print('drop前',len(dataset1))

dataset1.drop_duplicates(inplace=True)
# print('drop后',len(dataset1))
dataset2.drop_duplicates(inplace=True)
dataset12 = pd.concat([dataset1,dataset2],axis=0)
dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','coupon_id','day_gap_after'],axis=1)      
                                         
dataset3.drop_duplicates(inplace=True)                       
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]  # shape:112803 x 3
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)  # shape:112803 x 52

dataTrain = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataTest = xgb.DMatrix(dataset3_x)

#性能评价函数
def myauc(test):
    testgroup = test.groupby(['coupon_id'])
    aucs = []
    for i in testgroup:
        tmpdf = i[1] 
        if len(tmpdf['label'].unique()) != 2:
            continue
        fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred'], pos_label=1)
        aucs.append(auc(fpr,tpr))
    return np.average(aucs)

print('训练集：', len(dataset12_x),'\t测试集：',len(dataset3))
# print('数据集信息：\n',dataset12_x.info())
print('缺失值信息： ',dataset12_x.isnull().sum())

E:\D\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
E:\D\Anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


训练集： 382906 	测试集： 112803
缺失值信息：  discount_rate                                        0
distance                                         43656
day_of_month                                         0
days_distance                                        0
discount_man                                      7949
discount_jian                                     7949
is_man_jian                                          0
total_sales                                       6682
sales_use_coupon                                  5701
total_coupon                                      5701
merchant_min_distance                            72780
merchant_max_distance                            72780
merchant_mean_distance                           72780
merchant_median_distance                         72780
merchant_coupon_transfer_rate                    35922
coupon_rate                                       6682
count_merchant                                  172046
user_min_distance               

In [2]:
print(dataset3_x)
print(dataset3_preds)

        discount_rate  distance  day_of_month  days_distance  discount_man  \
0            0.833333       1.0            12             12          30.0   
1            0.833333       NaN             6              6          30.0   
2            0.900000       5.0            27             27         200.0   
3            0.900000       5.0            27             27         100.0   
4            0.966667       2.0             8              8          30.0   
5            0.833333       0.0             6              6          30.0   
6            0.833333       0.0            16             16          30.0   
7            0.833333       1.0            27             27          30.0   
8            0.833333      10.0            29             29          30.0   
9            0.833333       1.0             3              3          30.0   
10           0.833333       3.0            16             16          30.0   
11           0.833333       0.0            12             12    

In [3]:
# import os
# params={'booster':'gbtree',
# 	    'objective': 'rank:pairwise',
# 	    'eval_metric':'auc',
# 	    'gamma':0.1,
# 	    'min_child_weight':1.1,
# 	    'max_depth':5,
# 	    'lambda':10,
# 	    'subsample':0.7,
# 	    'colsample_bytree':0.7,
# 	    'colsample_bylevel':0.7,
# 	    'eta': 0.01,
# 	    'tree_method':'exact',
# 	    'seed':0,
# 	    'nthread':12
# 	    }
# watchlist = [(dataTrain,'train')]

# if not os.path.exists('../Data/model/xgbmodel'):
#     model = xgb.train(params,dataTrain,num_boost_round=3500,evals=watchlist)
#     model.save_model('../Data/model/xgbmodel')
# model=xgb.Booster(params)
# model.load_model('../Data/model/xgbmodel') 

# #predict test set 
# dataset3_preds1 = dataset3_preds
# dataset3_preds1['label'] = model.predict(dataTest)
# #标签归一化在[0，1]原作者代码这里有错
# #修改前
# #dataset3_preds.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds.label)
 
# #修改后
# dataset3_preds1.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds1.label.reshape(-1,1))
# dataset3_preds1.sort_values(by=['coupon_id','label'],inplace=True)
# dataset3_preds1.to_csv("../Data/prediction/xgb_preds.csv",index=None,header=None)
# print(dataset3_preds1.describe())

In [4]:
# model=xgb.Booster()
# model.load_model('../Data/model/xgbmodel') 

# temp = dataset12[['coupon_id','label']].copy()
# temp['pred'] =model.predict(xgb.DMatrix(dataset12_x))
# temp.pred = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(temp['pred'].values.reshape(-1,1))
# print(myauc(temp))

In [5]:
import os
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

# cvresult = xgb.cv(params, dataTrain, num_boost_round=10000, nfold=5, metrics='auc', seed=0, callbacks=[
#             xgb.callback.print_evaluation(show_stdv=False),
#             xgb.callback.early_stop(50)
#         ])
# num_round_best = cvresult.shape[0] - 1
# print('Best round num: ', num_round_best)
num_round_best = 7257   # 没有训练得到，随意新加的数据


watchlist = [(dataTrain,'train')]

if not os.path.exists('../Data/model/xgbmodel1_5_23'):
    model1 = xgb.train(params,dataTrain,num_boost_round=num_round_best,evals=watchlist)

    model1.save_model('../Data/model/xgbmodel1_5_23')
print('------------------------train done------------------------------')

------------------------train done------------------------------


In [6]:
model1=xgb.Booster()
model1.load_model('../Data/model/xgbmodel1_5_23') 

temp = dataset12[['coupon_id','label']].copy()
temp['pred'] =model1.predict(xgb.DMatrix(dataset12_x))
temp.pred = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(temp['pred'].values.reshape(-1,1))
print(myauc(temp))


0.7873190604970809


In [7]:
dataset3_preds2 = dataset3_preds
print(dataset3_preds2)
print(dataset3_x)
dataset3_preds2['label'] = model1.predict(dataTest)
dataset3_preds2.label = MinMaxScaler(copy=True,feature_range=(0,1)).fit_transform(dataset3_preds2.label.values.reshape(-1,1))
dataset3_preds2.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds2.to_csv("../Data/prediction/xgb_preds1_5_23.csv",index=None,header=None)
print(dataset3_preds2.describe())

        user_id  coupon_id  date_received
0       4129537       9983       20160712
1       6949378       3429       20160706
2       2166529       6928       20160727
3       2166529       1808       20160727
4       6172162       6500       20160708
5       4005121       9983       20160706
6       4347394       9983       20160716
7       3094273      13602       20160727
8       5139970       9983       20160729
9       3237121      13602       20160703
10      6224386       9983       20160716
11      6488578      13602       20160712
12      4164865       9983       20160703
13      4164865       8059       20160706
14      5468674       9983       20160713
15      6258178       9144       20160706
16      3659521       7341       20160727
17      3659521      13181       20160717
18      3659521      13602       20160718
19      7333378      13602       20160704
20      7333378        785       20160727
21      4454914       2978       20160711
22      6817282       8375       2

E:\D\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
E:\D\Anaconda\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
E:\D\Anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


            user_id      coupon_id  date_received          label
count  1.128030e+05  112803.000000   1.128030e+05  112803.000000
mean   3.684618e+06    9064.658006   2.016072e+07       0.348787
std    2.126358e+06    4147.283515   9.017693e+00       0.122218
min    2.090000e+02       3.000000   2.016070e+07       0.000000
25%    1.843824e+06    5035.000000   2.016071e+07       0.274075
50%    3.683073e+06    9983.000000   2.016072e+07       0.329727
75%    5.525176e+06   13602.000000   2.016072e+07       0.409607
max    7.361024e+06   14045.000000   2.016073e+07       1.000000


In [8]:
feature_score = model1.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)#value逆序排序

fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
 
with open('../Data/xgb_feature_score_5_23.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)